# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/midfielder/midfielder_po.pickle','rb') as f:
    midfielder_po = pickle.load(f)

midfielder_model = load_model('./models/midfielder/midfielder_nn.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']

In [5]:
# Goal_Shoot_linear
midfielder_df['GSL'] = (midfielder_df['경기당 유효 슈팅/90']+0.5) + (midfielder_df['경기당 슈팅/90']) + (midfielder_df['골']*1.3)

In [6]:
# PSTL = Pass sucess try linear

midfielder_df['PSTL'] = (midfielder_df['경기당 패스 성공/90']) + (midfielder_df['경기당 패스 시도/90']*1.15)

In [7]:
independent_value ={'GSL','PSTL','공중볼시도/90','기회 창출/90','도움','드리블 성공/90','반칙 수/90','키패스/90','태클 성공/90'}

dependent_value = {'평균획득승점/경기'}

In [8]:
midfielder_df.loc[:,independent_value] = midfielder_po.transform(midfielder_df.loc[:,independent_value])

In [9]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [10]:
midfielder_df['예측PPG']=midfielder_model.predict(midfielder_df.loc[:,independent_value])

In [11]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,GSL,PSTL,예측PPG
50,João Cancelo,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,1,5.199338,0,19,7,...,34.75,-1.010382,-0.104740,-5.199338,5.38,7.84,1.73,2.113083,5.199338,1.171221
22,Ricardo Pereira,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,4,5.199338,0,21,11,...,35.61,-0.781626,-0.105021,-5.199338,5.38,8.00,1.64,5.199338,5.199338,1.170648
16,Harvey Barnes,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,14,5.199338,0,22,14,...,30.42,-0.744694,-0.106679,-5.199338,4.05,5.62,1.45,5.199338,5.199338,1.162560
31,Trent Alexander-Arnold,2021-2022,미드필더,리버풀,잉글랜드 프리미어 리그,2,5.199338,0,25,10,...,36.69,-1.031509,-0.106650,-5.199338,5.97,9.22,1.85,5.199338,5.199338,1.147000
128,Jorginho,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,2,5.199338,0,32,10,...,45.86,-0.922330,0.217463,-5.199338,4.07,1.16,2.08,5.199338,5.199338,1.131540


In [12]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

1.1185829639434814

In [13]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [14]:
midfielder_df.loc[:,independent_value] = midfielder_po.inverse_transform(midfielder_df.loc[:,independent_value])

In [15]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)

In [16]:
midfielder_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,골,도움,WAR
0,João Cancelo,2021-2022,1,5.17,1.052638
1,Ricardo Pereira,2021-2022,4,5.17,1.052065
2,Harvey Barnes,2021-2022,14,5.17,1.043977
3,Trent Alexander-Arnold,2021-2022,2,5.17,1.028417
4,Jorginho,2021-2022,2,5.17,1.012957
5,Ainsley Maitland-Niles,2021-2022,1,5.17,1.012265
6,Vitaly Janelt,2021-2022,0,5.17,1.003176
7,Lucas Digne,2021-2022,0,5.00,1.002527
8,Christian Nørgaard,2021-2022,1,5.17,1.001946
9,Jordan Henderson,2021-2022,1,5.17,1.001754
